In [1]:
import lstm

We will start by defining a simple wrapper class that contains the entire parametrization of the  LSTM network.
This is implemented in `lstm.Parameters`.

In [2]:
%psource lstm.Parameters

class Parameters:
    def __init__(self, event_size: int, hidden_size: int):
        # Weights and biases for the sigmoid "f-function"
        self.event_forget_weights = np.zeros((hidden_size, event_size))
        self.event_forget_bias = 0
        self.hidden_forget_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_forget_bias = 0

        # Weights and biases for the sigmoid "i-function"
        self.event_update_weights = np.zeros((hidden_size, event_size))
        self.event_update_bias = 0
        self.hidden_update_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_update_bias = 0

        # Weights and biases for the tanh "C-bar-function"
        self.event_candidate_weights = np.zeros((hidden_size, event_size))
        self.event_candidate_bias = 0
        self.hidden_candidate_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_candidate_bias = 0

        # Weights and biases for the "o-function"
        self.event_output_weight

We will now instantiate this class.
For our dummy example, we will set the input size to 3 and the hidden size to 3 as well.

In [3]:
EVENT_SIZE, HIDDEN_SIZE = 3, 3
parameters = lstm.Parameters(event_size=EVENT_SIZE, hidden_size=HIDDEN_SIZE)

The forget gate is implemented as follows:

In [4]:
%psource lstm.forget_gate

def forget_gate(event, hidden_state, prev_cell_state, parameters):
    """Forget gate deciding how much of the previous cell state to keep."""
    forget_hidden = (
        parameters.hidden_forget_weights @ hidden_state
        + parameters.hidden_forget_bias
    )
    forget_event = (
        parameters.event_forget_weights @ event
        + parameters.event_forget_bias
    )
    # Values between zero and one indicating how much to forget
    forgetter = sigmoid(forget_hidden + forget_event)

    # Return the state that should be kept
    kept_state = forgetter * prev_cell_state
    return kept_state


Let's assume that we have an existing hidden state of $\vec{h} = [0, 0, 10]$, a previous cell state of $\vec{C} = [1, 1, 1]$, and a new event $\vec{x} = [10, 0, 0]$.
What happens in the forget gate with the current parametrization?

In [5]:
import numpy as np

event = np.array([10, 0, 0])
hidden_state = np.array([0, 0, 10])
prev_cell_state = np.array([1, 1, 1])
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.5, 0.5, 0.5])

Since we only have zero-weights, and zero biases, the new cell state has become $[0.5, 0.5, 0.5]$ since $\mathrm{sigmoid}(1) = 0.5$.
Let's change the weight matrices to identity matrices:

In [6]:
parameters.event_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.9999546, 0.5      , 0.9999546])

Changing both weight matrices to the identity matrices, the previous hidden state and the new event has influenced the new cell state.
We can also let the new event influence the cell state, and the hidden state be ignored completely:

In [7]:
parameters.event_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_forget_weights = np.zeros((HIDDEN_SIZE, EVENT_SIZE))
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.9999546, 0.5      , 0.5      ])

We see that the first index has been changed from $0.5$ to $\approx 1$, while the third index is still $0.5$ as expected.

---

The input gate is implemented as follows:

In [8]:
%psource lstm.input_gate

def input_gate(event, hidden_state, parameters):
    """Input gate deciding how to update the cell state."""
    # We have certain candidates from the new event and the hidden state
    # we would like to update the cell state with
    hidden_candidates = (
        parameters.hidden_candidate_weights @ hidden_state
        + parameters.hidden_candidate_bias
    )
    event_candidates = (
        parameters.event_candidate_weights @ event
        + parameters.event_candidate_bias
    )

    # We must also determine how much to weigh these updates
    event_update = (
        parameters.event_update_weights @ event
        + parameters.event_update_bias
    )
    hidden_update = (
        parameters.hidden_update_weights @ hidden_state
        + parameters.hidden_update_bias
    )

    # Finally returning the update
    return (
        sigmoid(event_update + hidden_update)
        * tanh(event_candidates + hidden_candidates)
    )


We try to evaluate this gate with the default $0$ weights.

In [9]:
lstm.input_gate(event=event, hidden_state=hidden_state, parameters=parameters)

array([0., 0., 0.])

This time we get $\vec{0}$ as the output from the input gate, due to the $\tanh$ activation function being zero for zero input.
Let's see what happens if we set the weight matrix for the hidden state to the identity matrix, but leaving the event weights as zero.

In [10]:
parameters.hidden_candidate_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_update_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
lstm.input_gate(event=event, hidden_state=hidden_state, parameters=parameters)

array([0.       , 0.       , 0.9999546])

The resulting update weighs the hidden state and ignores the new event data completely, as expected.
The new cell state is the sum of the forget gate output and the input gate output, as follows:

In [11]:
%psource lstm.cell_state

def cell_state(forget_gate_output, input_gate_output):
    """
    New cell state, a combination of the partially forgotten cell state
    and the newly proposed state.
    """
    return forget_gate_output + input_gate_output


---

The output of the LSTM cell is a combination of the new event, the hidden state, and the cell state.
It is implemented as follows:

In [13]:
%psource lstm.output_gate

def output_gate(event, hidden_state, cell_state, parameters):
    """Decide what to output from the LSTM cell."""
    hidden_output = (
        parameters.hidden_output_weights @ hidden_state
        + parameters.hidden_output_bias
    )
    event_output = (
        parameters.event_output_weights @ event
        + parameters.event_output_bias
    )
    return (
        sigmoid(event_output + hidden_output)
        * tanh(cell_state)
    )


For the given LSTM network we have implemented, the event input size is $x \times 1$, and the hidden state is of size $h \times 1$.
The output of the LSTM cell becomes $h \times 1$, which is not necessarily optimal for a given problem.
The output of the LSTM cell therefore often passed to final, fully connected layer with an output of more optimal dimensions.
We have opted to not implement this last part, since it is not a core, new concept of LSTM cells.